In [1]:
from os import listdir
from PIL import Image
from numpy import asarray, vstack, savez_compressed
from tensorflow.keras.preprocessing.image import img_to_array, load_img, smart_resize, array_to_img
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [2]:
# crop and scale scale image
def scale(path, height, width):
    
    # load a scaled version of the image
    img = load_img(path, target_size=(height,width))
    
    # set the box based which side is larger
    if height < width:
        left = (width - 256) / 2
        top = 0
        right = width - ((width - 256) / 2)
        bottom = height
        
        img = img.crop((left,top,right,bottom))
    elif width < height:
        left = 0
        top = (height - 256) / 2
        right = width
        bottom = height - ((height - 256) / 2)
        
        img = img.crop((left,top,right,bottom))
    
    return img

In [10]:
# function to load, rescale and crop a dataset into 256x256
def load_scale_crop(path, stop=0):
    
    data_list = list()
    count = 0
    
    # enumerate filenames in directory
    for filename in listdir(path):
        
        if filename == 'desktop.ini':
            continue
        
        if count >= stop and stop != 0:
            break
        
        img = load_img(path+filename)
        img = img_to_array(img)
        
        if img.shape[0] < img.shape[1]:
            height = int(img.shape[0] / (img.shape[0] / 256))
            width = int(img.shape[1] / (img.shape[0] / 256))
            
            formated = scale(path+filename, height, width)
            
            formated = img_to_array(formated)
            
            data_list.append(formated)
        elif img.shape[1] < img.shape[0]:
            height = int(img.shape[0] / (img.shape[1] / 256))
            width = int(img.shape[1] / (img.shape[1] / 256))
            
            formated = scale(path+filename, height, width)
            
            formated = img_to_array(formated)
            
            data_list.append(formated)
        else:
            height = int(img.shape[0] / (img.shape[0]/ 256))
            width = int(img.shape[1] / (img.shape[1]/ 256))
            
            formated = load_img(path+filename, target_size=(height,width))
            
            formated = img_to_array(formated)
            
            data_list.append(formated)
            
        count += 1
            
    return asarray(data_list)

In [4]:
def resize_crop_save(old_path, new_path, filename, stop=0):
    
    data = load_scale_crop(old_path, stop)
    
    count = 0
    
    for array in data:
        
        if count >= 300:
            break
            
        img = array_to_img(array)
        img.save(f'{new_path}{count}{filename}.jpg', 'JPEG')
        count += 1

In [9]:
listdir('people_bjarki/')[0]

'desktop.ini'

In [11]:
resize_crop_save('train2014_usable/', 'data/photo/', '2014_usab')

In [12]:
type(test)

PIL.Image.Image

In [4]:
# stop condition
stop = len(listdir('Munch_Cleaned_Types/'))

# dataset path
path = 'Munch_Cleaned_Types/'

# load, split and stack dataset B
dataB = load_scale_crop(path, stop)
trainB, testB = train_test_split(dataB, test_size=0.3)
dataB = vstack((trainB, testB))
print('DataB: ', dataB.shape)

# update path
path = 'data/photo_jpg/'

# load, split and stack dataset A
dataA = load_scale_crop(path, stop)
trainA, testA = train_test_split(dataA, test_size=0.3)
dataA = vstack((trainA, testA))
print('DataA: ', dataA.shape)

DataB:  (1514, 256, 256, 3)
DataA:  (1514, 256, 256, 3)


In [5]:
# save as compressed numpy array
filename = 'photo2munch_cleaned_sametype_256.npz'
savez_compressed(filename, a=dataA, b=dataB)
print('Saved dataset: ', filename)

Saved dataset:  photo2munch_cleaned_sametype_256.npz
